In [2]:
!pip install -U memory_profiler

  Obtaining dependency information for memory_profiler from https://files.pythonhosted.org/packages/49/26/aaca612a0634ceede20682e692a6c55e35a94c21ba36b807cc40fe910ae1/memory_profiler-0.61.0-py3-none-any.whl.metadata


In [4]:
!pip install time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [8]:
import json
import numpy as np
from sklearn.model_selection import train_test_split #Used for data spliting
import tensorflow.keras as keras
import matplotlib.pyplot as plt #For plots

'''
#Inference Imports
from IPython import display
import os
import pathlib
import tensorflow as tf
'''


DATASET_PATH = "data.json" #The path to the dataset


def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)
        
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

def prepare_datasets(test_size, validation_size):
    #Load data
    X, y = load_data(DATASET_PATH)
    
    #Create train/test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size)
    
    
    #Create the train/Validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)
    
    #CNNs expect 3D arrays (dim, dim, channel) --> eg. (64, 64, 3)
    X_train = X_train[..., np.newaxis] #This is a 4D array --> (num_samples, 64, 64, 3)
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test



def build_model(input_shape):
    #Create model
    model = keras.Sequential()
    
    #1st Conv Layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2,2), padding='same')) #zero padding, same all round
    model.add(keras.layers.BatchNormalization()) #Helps the models converge faster
    
    #2nd Conv Layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((3, 3), strides=(2,2), padding='same')) #zero padding, same all round
    model.add(keras.layers.BatchNormalization()) #Helps the models converge faster
    
    #3rd Conv Layer
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPool2D((2, 2), strides=(2,2), padding='same')) #zero padding, same all round
    model.add(keras.layers.BatchNormalization()) #Helps the models converge faster
    
    
    #Flatten the output and feed to Dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3)) #To avoid Overfitting issues we use dropout
    
    
    #Output Layer - Softmax function (classifier)
    model.add(keras.layers.Dense(10, activation='softmax'))
    
    
    return model


def predict(model, X, y):
    X = X[np.newaxis, ...]
    
    #Prediction is a 2D array
    prediction = model.predict(X) #X - -> (1, 130, 13, 1)
    
    #Extract the index with the max value
    predicted_index = np.argmax(prediction, axis=1)
    
    print("Expected index: {}, Predicted index: {}".format(y, predicted_index ))


if __name__ == "__main__":
    #Create train, Validation and Test Sets
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2) #0.2 means 20percent will be for validation
    
    #Build the CNN net
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    model = build_model(input_shape)
    
    #Compile the CNN Network
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    
    #Train the Network
    #model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=10) #Initially 30 epoch
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=100) #Initially 30 epoch, try epoch 10 too
    
    #Evaluate the CNN on the dataset
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy on test set is: {}".format(test_accuracy))
    
    
    #Make Predictions on a sample
    X = X_test[100]
    y = y_test[100]
    
    
    predict(model, X, y)
    
    
    
    #Save Model
    
    model.save('saved_conv_model/my_model_profiled')#This directory creates itself. You don't have to create one b4 running
    
    # Save the entire model to a HDF5 file.
    # The '.h5' extension indicates that the model should be saved to HDF5.
    model.save('saved_conv_model/my_model_profiled.h5')
    
    
    
    #Plot
    metrics = history.history
    plt.figure(figsize=(16,6))
    fig = plt.figure(figsize=(16,6)) #To save figure (loss and accuracy)
    plt.subplot(1,2,1)
    plt.plot(history.epoch, metrics['loss'], metrics['val_loss'], marker='*')
    plt.legend(['loss', 'val_loss'])
    plt.ylim([0, max(plt.ylim())])
    plt.xlabel('Epoch')
    plt.ylabel('Loss [Cross-Entropy]')
    
    plt.subplot(1,2,2)
    plt.plot(history.epoch, 100*np.array(metrics['accuracy']), 100*np.array(metrics['val_accuracy']), marker='*')
    plt.legend(['accuracy', 'val_accuracy'])
    plt.ylim([0, 100])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy - Percent [%]')
    
    plt.savefig('Accuracy_loss_profiled.png')
    

TypeError: 'module' object is not callable